## Instance Struture

In [1]:
mutable struct Instance
    n :: Int64
    D
    l
    function Instance(s)
        D = zeros(1,1)
        nNodes = 0
        l = 0
        open(s) do f
            line = 1
            lineContent = readline(f)
            lineSplited = split(lineContent, " ")
            nNodes = parse(Int64, lineSplited[1])
            l = parse(Int64, lineSplited[2])
            D = zeros(nNodes,nNodes)
            while ! eof(f)
                lineContent = readline(f)
                lineSplited = split(lineContent, " ")
                for col = 1:nNodes
                    D[line,col] = parse(Float64, lineSplited[col])
                end
                
                line += 1
            end
        end
        new(nNodes,D,l)
    end
end

## Solution Structure


In [2]:
mutable struct Solution
    π :: Array{Int64,1} ## solução: uma permutação de cidades
    v :: Float64        ## valor da solução: distância total
    Solution(s::Solution) = new(copy(s.π),s.v)
    Solution(π,v) = new(copy(π),v)
end
Base.copy(s::Solution) = Solution(s)

 ## Nearest Neighbor

In [3]:
function nearestNeighbor(I::Instance)
    s = Solution([1],0.0)
    free=trues(I.n)
    free[1]=false
    for k=2:I.n
      f=findall(free)
      j=f[findmin(I.D[s.π[end],f])[2]]
    #       s.v += I.D[s.π[end],j]
      push!(s.π,j)
      free[j]=false
    end
    for lookahead=1:I.l
        for i=1:I.n
            s.v += I.D[s.π[i],s.π[((i+lookahead-1)%(I.n))+1]]
        end
    end
#     push!(s.π,1)
    s
end

nearestNeighbor (generic function with 1 method)

## Random

In [4]:
function random(I::Instance,k)
    s = Solution([1],0.0)
    free=trues(I.n)
    free[1]=false
    for i=2:I.n
         f=findall(free)
         ncand=min(k,I.n-i+1)
         j=f[sortperm(I.D[s.π[end],f])[rand(1:ncand)]]
#          s.v += I.D[s.π[end],j]
         push!(s.π,j)
         free[j]=false
    end
    for lookahead=1:I.l
        for i=1:I.n
#             println("$(s.π[i]) - $(s.π[((i+lookahead-1)%(I.n))+1])")
#             println("$(I.D[i,((i+lookahead-1)%(I.n))+1])")
            s.v += I.D[s.π[i],s.π[((i+lookahead-1)%(I.n))+1]]
        end
    end
#     push!(s.π,1)
    s
end

random (generic function with 1 method)

## Local Search with Adjacent Cities Swap

In [5]:
function localSearchAdjacentCitiesSwap(I::Instance, s₀::Solution)
    s = copy(s₀)
    i=1
    v=0.0
    while i<I.n
        i=2
        while i<I.n
            
            
#             v=s.v-I.D[s.π[a],s.π[a+1]]-I.D[s.π[b],s.π[b+1]]+I.D[s.π[a],s.π[b]]+I.D[s.π[a+1],s.π[b+1]]            
#             v=s.v-I.D[s.π[i-1],s.π[i]]-I.D[s.π[i+1],s.π[i+2]]+I.D[s.π[i-1],s.π[i+1]]+I.D[s.π[i],s.π[i+2]]
            newπ = s.π 
            aux = newπ[i] #swap
            newπ[i] = newπ[i+1]
            newπ[i+1] = aux

            v = 0
            for lookahead=1:I.l
                for i=1:I.n
                    v += I.D[newπ[i],newπ[((i+lookahead-1)%(I.n))+1]]
                end
            end
            
            
            
            v<s.v && break ## first improvement
            i += 1
        end
        if i<I.n
            s.π[i],s.π[i+1] = s.π[i+1],s.π[i]
            s.v = v
        end
    end
    s
end

localSearchAdjacentCitiesSwap (generic function with 1 method)

## Local Search with 2-exchange

In [6]:
function localSearch2Exchange(I::Instance,s₀::Solution)
    s=copy(s₀)
    move=(1,1)
    v=0.0
    while move!=(0,0)
        move=(0,0)
        for a=1:I.n, b=(a+2):I.n
            a==1 && b==I.n && continue
            
            newπ = s.π 
            aux = newπ[a+1] #swap
            newπ[a+1] = newπ[b]
            newπ[b] = aux

            v = 0
            for lookahead=1:I.l
                for i=1:I.n
                    v += I.D[newπ[i],newπ[((i+lookahead-1)%(I.n))+1]]
                end
            end
            
#             v=s.v-I.D[s.π[a],s.π[a+1]]-I.D[s.π[b],s.π[b+1]]+I.D[s.π[a],s.π[b]]+I.D[s.π[a+1],s.π[b+1]]
        
            if v<s.v
                move=(a,b)
                break ## first improvement
            end
        end
        if move!=(0,0)
            ## reverse segment
            move=(move[1]+1,move[2])
            while move[1]<move[2]
                s.π[move[1]],s.π[move[2]] = s.π[move[2]],s.π[move[1]]
                move=(move[1]+1,move[2]-1)
            end
            s.v=v
        end
    end
    s
end

localSearch2Exchange (generic function with 1 method)

In [ ]:
I = Instance("instances/rml10.txt")
b = nearestNeighbor(I)
println("Vizinho mais próximo: $(b.v).")

for i = 1:100
    println("Iteração $i: $(b.v)")
    s = random(I,2)
    s = localSearchAdjacentCitiesSwap(I,s)
    if s.v < b.v
        b = copy(s)
    end
end
println("Best solution $(b.v).")

Vizinho mais próximo: 3.0128831e7.
Iteração 1: 3.0128831e7
Iteração 2: 2.6645558e7
Iteração 3: 2.6645558e7
Iteração 4: 2.6645558e7
Iteração 5: 2.6645558e7
Iteração 6: 2.5204235e7
Iteração 7: 2.5204235e7
Iteração 8: 2.4974404e7
Iteração 9: 2.4974404e7
Iteração 10: 2.4974404e7